# Random Forest
We'll try to load more features than in notebook 01: nucleus area central moment.

Accuracy 60% (but varies per run).

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn
print('sklearn',sklearn.__version__)

2022-05-25 15:16:12.136195
Python 3.8.10
sklearn 1.0.2


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
# Data straight from CellProfiler (rows for every patch image file)
BASE_DIR='/home/jrm/Adjeroh/Naved/Process100.class/'
CLASS_DIRS=['0','1','2','3','4','5']
NUM_CLASSES=len(CLASS_DIRS)
IMG_FN="Process100_Image.csv"
RBC_FN="Process100_MergeRBC.csv"
NUC_FN="Process100_Nucleus.csv"
CEL_FN="Process100_ExpandCells.csv"
TIS_FN="Process100_Tissue.csv"
# Saving our models
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.01/'

In [4]:
# show columns
df = pd.read_csv(BASE_DIR+CLASS_DIRS[0]+'/'+NUC_FN)
initial_cols = df.columns
for col in initial_cols:
    print(col)

ImageNumber
ObjectNumber
FileName_Tumor
PathName_Tumor
AreaShape_Area
AreaShape_BoundingBoxArea
AreaShape_BoundingBoxMaximum_X
AreaShape_BoundingBoxMaximum_Y
AreaShape_BoundingBoxMinimum_X
AreaShape_BoundingBoxMinimum_Y
AreaShape_Center_X
AreaShape_Center_Y
AreaShape_CentralMoment_0_0
AreaShape_CentralMoment_0_1
AreaShape_CentralMoment_0_2
AreaShape_CentralMoment_0_3
AreaShape_CentralMoment_1_0
AreaShape_CentralMoment_1_1
AreaShape_CentralMoment_1_2
AreaShape_CentralMoment_1_3
AreaShape_CentralMoment_2_0
AreaShape_CentralMoment_2_1
AreaShape_CentralMoment_2_2
AreaShape_CentralMoment_2_3
AreaShape_Compactness
AreaShape_ConvexArea
AreaShape_Eccentricity
AreaShape_EquivalentDiameter
AreaShape_EulerNumber
AreaShape_Extent
AreaShape_FormFactor
AreaShape_HuMoment_0
AreaShape_HuMoment_1
AreaShape_HuMoment_2
AreaShape_HuMoment_3
AreaShape_HuMoment_4
AreaShape_HuMoment_5
AreaShape_HuMoment_6
AreaShape_InertiaTensorEigenvalues_0
AreaShape_InertiaTensorEigenvalues_1
AreaShape_InertiaTensor_0_0
Ar

In [5]:
df = None
GOOD_COLS=['ImageNumber','AreaShape_Area','AreaShape_MeanRadius','Neighbors_NumberOfNeighbors_Expanded']
GOOD_COLS.extend([x for x in initial_cols if x.startswith( ('AreaShape_CentralMoment') ) ] )
type(GOOD_COLS)
GOOD_COLS

['ImageNumber',
 'AreaShape_Area',
 'AreaShape_MeanRadius',
 'Neighbors_NumberOfNeighbors_Expanded',
 'AreaShape_CentralMoment_0_0',
 'AreaShape_CentralMoment_0_1',
 'AreaShape_CentralMoment_0_2',
 'AreaShape_CentralMoment_0_3',
 'AreaShape_CentralMoment_1_0',
 'AreaShape_CentralMoment_1_1',
 'AreaShape_CentralMoment_1_2',
 'AreaShape_CentralMoment_1_3',
 'AreaShape_CentralMoment_2_0',
 'AreaShape_CentralMoment_2_1',
 'AreaShape_CentralMoment_2_2',
 'AreaShape_CentralMoment_2_3']

In [6]:
def nuc_prefix(col_name):
    return 'Nuc_'+col_name
def load_one_class(class_index):
    class_name=CLASS_DIRS[class_index]+'/'
    object_df = pd.read_csv(BASE_DIR+class_name+NUC_FN)
    print("Load %d nuclei." % len(object_df))
    object_df = object_df[GOOD_COLS]
    object_df.rename(nuc_prefix,axis='columns',inplace=True)
    patch_df = object_df.groupby(['Nuc_ImageNumber']).describe()
    # The random forest classifier cannot handle two-part column names, so convert them to string.    
    patch_df.columns=patch_df.columns.map('_'.join)
    patch_df.fillna(0,inplace=True)
    print("Load %d patches having nuclei." % len(patch_df))
    return patch_df
df = load_one_class(0)

Load 1826 nuclei.
Load 100 patches having nuclei.


In [7]:
df

,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_MeanRadius_count,Nuc_AreaShape_MeanRadius_mean,...,Nuc_AreaShape_CentralMoment_2_2_75%,Nuc_AreaShape_CentralMoment_2_2_max,Nuc_AreaShape_CentralMoment_2_3_count,Nuc_AreaShape_CentralMoment_2_3_mean,Nuc_AreaShape_CentralMoment_2_3_std,Nuc_AreaShape_CentralMoment_2_3_min,Nuc_AreaShape_CentralMoment_2_3_25%,Nuc_AreaShape_CentralMoment_2_3_50%,Nuc_AreaShape_CentralMoment_2_3_75%,Nuc_AreaShape_CentralMoment_2_3_max
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,313.000000,0.000000,313.0,313.00,313.0,313.00,313.0,1.0,3.387379,...,118865.676038,1.188657e+05,1.0,2.147786e+05,0.000000e+00,2.147786e+05,2.147786e+05,2.147786e+05,2.147786e+05,2.147786e+05
2,3.0,251.000000,79.680612,178.0,208.50,239.0,287.50,336.0,3.0,2.472544,...,169649.066602,2.023582e+05,3.0,-2.420412e+05,3.503882e+05,-6.461319e+05,-3.517955e+05,-5.745916e+04,-3.999592e+04,-2.253268e+04
3,5.0,302.000000,162.260593,182.0,212.00,260.0,271.00,585.0,5.0,3.278895,...,82919.818782,9.447153e+05,5.0,-7.222369e+03,6.457634e+04,-1.155200e+05,-1.109986e+01,9.074318e+03,1.237114e+04,5.797382e+04
4,29.0,314.137931,125.210030,177.0,233.00,262.0,340.00,635.0,29.0,3.072118,...,333522.028152,2.920010e+06,29.0,-3.307109e+05,3.650528e+06,-1.578665e+07,-1.332680e+05,6.622655e+03,1.571750e+05,5.995018e+06
5,32.0,355.937500,160.779339,180.0,243.00,331.0,375.25,832.0,32.0,3.188709,...,499219.000742,6.763447e+06,32.0,-1.417202e+06,5.749609e+06,-2.734124e+07,-3.124748e+05,-2.043129e+04,1.970674e+05,6.984380e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,2.0,306.500000,47.376154,273.0,289.75,306.5,323.25,340.0,2.0,3.069114,...,321135.044577,3.639205e+05,2.0,-5.673559e+05,8.901179e+05,-1.196764e+06,-8.820601e+05,-5.673559e+05,-2.526517e+05,6.205249e+04
97,32.0,394.750000,171.982745,189.0,270.50,352.5,480.75,812.0,32.0,3.372149,...,641790.350654,8.696358e+06,32.0,6.464846e+05,5.573771e+06,-5.065539e+06,-4.576935e+05,-6.460115e+04,5.213239e+04,2.816401e+07
98,23.0,318.565217,120.095729,183.0,244.50,285.0,373.00,633.0,23.0,3.056807,...,575493.366758,4.182120e+06,23.0,-1.555103e+06,5.952774e+06,-2.819849e+07,-2.617506e+05,1.182124e+04,4.924614e+04,1.027166e+06


In [8]:
df = None
def load_all_classes():
    X = load_one_class(0)
    size = len(X)
    y = np.zeros(size)
    for i in range(1,NUM_CLASSES):
        Xi = load_one_class(i)
        size = len(Xi)
        yi = np.ones(size) * i
        X = pd.concat( (X,Xi) )
        y = np.concatenate( (y,yi) )
    return X,y
X,y=load_all_classes()
X

Load 1826 nuclei.
Load 100 patches having nuclei.
Load 2435 nuclei.
Load 100 patches having nuclei.
Load 886 nuclei.
Load 100 patches having nuclei.
Load 1459 nuclei.
Load 100 patches having nuclei.
Load 1713 nuclei.
Load 100 patches having nuclei.
Load 3659 nuclei.
Load 100 patches having nuclei.


,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,Nuc_AreaShape_Area_std,Nuc_AreaShape_Area_min,Nuc_AreaShape_Area_25%,Nuc_AreaShape_Area_50%,Nuc_AreaShape_Area_75%,Nuc_AreaShape_Area_max,Nuc_AreaShape_MeanRadius_count,Nuc_AreaShape_MeanRadius_mean,...,Nuc_AreaShape_CentralMoment_2_2_75%,Nuc_AreaShape_CentralMoment_2_2_max,Nuc_AreaShape_CentralMoment_2_3_count,Nuc_AreaShape_CentralMoment_2_3_mean,Nuc_AreaShape_CentralMoment_2_3_std,Nuc_AreaShape_CentralMoment_2_3_min,Nuc_AreaShape_CentralMoment_2_3_25%,Nuc_AreaShape_CentralMoment_2_3_50%,Nuc_AreaShape_CentralMoment_2_3_75%,Nuc_AreaShape_CentralMoment_2_3_max
Nuc_ImageNumber,,,,,,,,,,,,,,,,,,,,,
1,1.0,313.000000,0.000000,313.0,313.00,313.0,313.00,313.0,1.0,3.387379,...,1.188657e+05,1.188657e+05,1.0,2.147786e+05,0.000000e+00,2.147786e+05,2.147786e+05,2.147786e+05,2.147786e+05,2.147786e+05
2,3.0,251.000000,79.680612,178.0,208.50,239.0,287.50,336.0,3.0,2.472544,...,1.696491e+05,2.023582e+05,3.0,-2.420412e+05,3.503882e+05,-6.461319e+05,-3.517955e+05,-5.745916e+04,-3.999592e+04,-2.253268e+04
3,5.0,302.000000,162.260593,182.0,212.00,260.0,271.00,585.0,5.0,3.278895,...,8.291982e+04,9.447153e+05,5.0,-7.222369e+03,6.457634e+04,-1.155200e+05,-1.109986e+01,9.074318e+03,1.237114e+04,5.797382e+04
4,29.0,314.137931,125.210030,177.0,233.00,262.0,340.00,635.0,29.0,3.072118,...,3.335220e+05,2.920010e+06,29.0,-3.307109e+05,3.650528e+06,-1.578665e+07,-1.332680e+05,6.622655e+03,1.571750e+05,5.995018e+06
5,32.0,355.937500,160.779339,180.0,243.00,331.0,375.25,832.0,32.0,3.188709,...,4.992190e+05,6.763447e+06,32.0,-1.417202e+06,5.749609e+06,-2.734124e+07,-3.124748e+05,-2.043129e+04,1.970674e+05,6.984380e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,47.0,574.255319,290.847448,177.0,305.50,534.0,746.00,1323.0,47.0,3.914974,...,3.215983e+06,1.482594e+07,47.0,4.941275e+06,2.987623e+07,-6.199312e+07,-9.914849e+05,-8.681557e+04,1.032628e+06,1.308756e+08
97,5.0,667.600000,270.346629,280.0,513.00,734.0,888.00,923.0,5.0,3.732732,...,1.147820e+07,1.909488e+07,5.0,-2.061896e+07,2.775551e+07,-6.763106e+07,-2.330142e+07,-9.215152e+06,-2.181614e+06,-7.655517e+05
98,50.0,549.580000,293.608798,181.0,347.00,440.0,728.75,1565.0,50.0,3.927443,...,2.411024e+06,3.545597e+07,50.0,9.128963e+06,5.008156e+07,-1.750266e+07,-8.587996e+05,1.036962e+04,7.239826e+05,3.139251e+08


In [9]:
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [10]:
Xtrain,Xvalid,ytrain,yvalid = train_test_split(X, y.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xtrain.shape,'ytrain',ytrain.shape,'non-zero:',np.count_nonzero(ytrain))
print('Xvalid',Xvalid.shape,'yvalid',yvalid.shape,'non-zero:',np.count_nonzero(yvalid))

Xtrain (450, 120) ytrain (450,) non-zero: 381
Xvalid (150, 120) yvalid (150,) non-zero: 119


In [11]:
ytrain

array([2., 4., 4., 0., 4., 5., 2., 4., 4., 5., 2., 1., 4., 2., 2., 5., 3.,
       2., 4., 3., 1., 1., 5., 0., 1., 4., 2., 2., 2., 3., 4., 2., 1., 5.,
       0., 5., 5., 3., 0., 4., 2., 5., 1., 0., 3., 4., 4., 4., 3., 2., 3.,
       1., 0., 3., 4., 0., 1., 1., 4., 2., 3., 5., 0., 5., 5., 0., 3., 5.,
       3., 0., 5., 3., 4., 4., 0., 1., 1., 2., 4., 5., 5., 0., 5., 0., 4.,
       3., 4., 4., 4., 1., 2., 3., 4., 5., 3., 5., 2., 4., 3., 1., 3., 0.,
       4., 0., 5., 2., 3., 5., 4., 1., 1., 5., 5., 2., 4., 5., 4., 5., 1.,
       1., 1., 5., 5., 3., 5., 2., 3., 4., 2., 2., 4., 2., 2., 2., 3., 2.,
       4., 3., 3., 4., 3., 5., 0., 2., 0., 1., 3., 3., 2., 3., 3., 2., 2.,
       0., 4., 4., 1., 2., 1., 3., 2., 3., 1., 0., 4., 0., 2., 3., 4., 0.,
       3., 4., 4., 1., 0., 1., 3., 1., 4., 0., 1., 2., 4., 0., 4., 4., 1.,
       1., 2., 1., 0., 3., 0., 5., 4., 5., 4., 1., 1., 5., 0., 4., 0., 0.,
       4., 2., 1., 0., 4., 3., 3., 1., 5., 2., 1., 5., 4., 4., 3., 1., 3.,
       4., 0., 5., 2., 0.

In [12]:
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model.fit(self.Xtr,self.ytr)
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  # bug fix
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

In [13]:
print(datetime.datetime.now())
print('Train on all Features')
rf1 = RF_Util()
rf1.set_train(Xtrain,ytrain)
rf1.set_validation(Xvalid,yvalid)
rf1.fit()
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())
print('The impurity-based feature importances.')
top = rf1.important_features()
top.head()

2022-05-25 15:16:21.863779
Train on all Features
Accuracy: 60.666666666666664
Confusion:
[[23  0  2  3  3  0]
 [ 2 13  2  0  6  1]
 [ 1  0 20  2  4  0]
 [ 0  5  6  5  5  0]
 [ 4  1  5  0  9  0]
 [ 0  2  4  0  1 21]]
The impurity-based feature importances.


,0,1
0,0.023251,Nuc_AreaShape_MeanRadius_mean
1,0.019809,Nuc_AreaShape_CentralMoment_0_0_75%
2,0.019547,Nuc_AreaShape_CentralMoment_0_3_count
3,0.019132,Nuc_AreaShape_Area_75%
4,0.018766,Nuc_AreaShape_CentralMoment_0_0_mean
